In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import pickle
import glob

### Functions

In [3]:
def read_annotation(annotation_path):
    lines = []
    with open(annotation_path) as f:
        lines.append(f.read().splitlines() )
    f.close()
    #lines = np.sort(lines)
    lines = np.hstack(lines)
    return lines

def generate_train_test_list(lists):
    train_list = []
    test_list = []
    for i in range(len(lists)):
        lines = read_annotation(lists[i])
        for line in lines:
            file_name, flag = line.split(' ')
            if flag == '1':
                train_list.append(file_name.split('.')[0])
            elif flag == '2':
                test_list.append(file_name.split('.')[0])
    return train_list,test_list

import scipy.io
def read_pose(path):
    mat = scipy.io.loadmat(path)
    poses = np.round(mat['pos_world'],3).swapaxes(0,2)
    return poses

def generate_pose_label(pose_list,train_list,test_list):
    train = {}
    train['pose'] = []
    train['label'] = []
    test = {}
    test['pose'] = []
    test['label'] = []
    for i in range(len(pose_list)):
        label = pose_list[i].split('/')[-2]
        pose_path = pose_list[i]+'/joint_positions.mat'
        pose = read_pose(pose_path)
        file = pose_list[i].split('/')[-1]
        if file in train_list:
            train['label'].append(label)
            train['pose'].append(pose)
        elif file in test_list:
            test['label'].append(label)
            test['pose'].append(pose)
    return train,test

### Config settings

In [4]:
class Config():
    def __init__(self):        
        self.frame_l = 32 # the length of frames
        self.joint_n = 15 # the number of joints
        self.joint_d = 2 # the dimension of classes 
        self.data_dir = '/mnt/nasbi/no-backups/datasets/pose_action/JHMDB/'
        self.save_dir = '/mnt/nasbi/homes/fan/projects/action/skeleton/data/JHMDB/'
C = Config()

### There are 3 ways of splitting the ground-truth pose data

In [5]:
GT_split_lists = glob.glob(C.data_dir + 'GT_splits/*.txt')
GT_pose_list = glob.glob(C.data_dir + 'GT_joint_positions/*/*')

In [6]:
GT_lists_1 = []
GT_lists_2 = []
GT_lists_3 = []
for file in GT_split_lists:
    if file.split('/')[-1].split('.')[0].split('_')[-1] == 'split1':
        GT_lists_1.append(file) 
    elif file.split('/')[-1].split('.')[0].split('_')[-1] == 'split2':
        GT_lists_2.append(file)
    elif file.split('/')[-1].split('.')[0].split('_')[-1] == 'split3':
        GT_lists_3.append(file)
       

## Processing each spliting method

In [7]:
GT_train_list_1,GT_test_list_1 = generate_train_test_list(GT_lists_1)

GT_train_1,GT_test_1 = generate_pose_label(GT_pose_list,GT_train_list_1,GT_test_list_1)

pickle.dump(GT_train_1, open(C.save_dir+"GT_train_1.pkl", "wb"))
pickle.dump(GT_test_1, open(C.save_dir+"GT_test_1.pkl", "wb"))


In [8]:
GT_train_list_2,GT_test_list_2 = generate_train_test_list(GT_lists_2)

GT_train_2,GT_test_2 = generate_pose_label(GT_pose_list,GT_train_list_2,GT_test_list_2)

pickle.dump(GT_train_2, open(C.save_dir+"GT_train_2.pkl", "wb"))
pickle.dump(GT_test_2, open(C.save_dir+"GT_test_2.pkl", "wb"))


In [9]:
GT_train_list_3,GT_test_list_3 = generate_train_test_list(GT_lists_3)

GT_train_3,GT_test_3 = generate_pose_label(GT_pose_list,GT_train_list_3,GT_test_list_3)

pickle.dump(GT_train_3, open(C.save_dir+"GT_train_3.pkl", "wb"))
pickle.dump(GT_test_3, open(C.save_dir+"GT_test_3.pkl", "wb"))
